In [1]:
#fine tuning SAM2 with Cityscapes dataset
import os, sys
import numpy as np
KD_path = "/home/avalocal/thesis23/KD"
sys.path.append(KD_path)

In [2]:
'''scratch:
  resolution: 1024
  train_batch_size: 1
  num_train_workers: 10
  num_frames: 8
  max_num_objects: 3
  base_lr: 5.0e-6
  vision_lr: 3.0e-06
  phases_per_epoch: 1
  num_epochs: 40'''



'scratch:\n  resolution: 1024\n  train_batch_size: 1\n  num_train_workers: 10\n  num_frames: 8\n  max_num_objects: 3\n  base_lr: 5.0e-6\n  vision_lr: 3.0e-06\n  phases_per_epoch: 1\n  num_epochs: 40'

In [3]:
#dataset 
#images 
#annorations: labelIds
#prompts: bboxes from labelIds (might consider or not)
import os, sys, glob
from PIL import Image
import torch
from torch.utils.data import Dataset
from cityscapes_original import CityscapesSV
import torch.optim as optim
from torch.utils.data import DataLoader
from loss_fns import MultiStepMultiMasksAndIous, sigmoid_focal_loss, dice_loss
from torchvision.transforms import ToTensor

#dataset and dataloaders
# train_dataset = CityscapesSV(root_dir="/media/avalocal/T7/pardis/pardis/perception_system/datasets/cityscapes", split='train')
train_dataset = CityscapesSV(root_dir="/media/avalocal/T7/pardis/pardis/perception_system/datasets/cityscapes/cityscapes_few_shot_instance", split='train')

print(len(train_dataset))
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

train_batch_size = 4
num_train_workers = 4

# image, instance, ids,  bbox, num_objects, file_name



/home/avalocal/miniconda3/envs/KD/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (None)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn(
/home/avalocal/miniconda3/envs/KD/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


83


In [4]:
from hydra import initialize_config_dir, compose
from omegaconf import DictConfig
import os
#fine-tuning sam2
ckpt_path = '/media/avalocal/T7/pardis/pardis/perception_system/ckpt/sam2.pth'

from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
import torch.nn as nn

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class SAM2Model(nn.Module):   #used for teacher model of depth prediction
    def __init__(self):
        super(SAM2Model, self).__init__()

        # sam2_checkpoint = os.path.join(current_dir, "sam2/checkpoints/sam2.1_hiera_large.pt")
        sam2_checkpoint = "/home/avalocal/thesis23/KD/sam2/checkpoints/sam2.1_hiera_large.pt"
        # model_cfg = "sam2.1/sam2.1_hiera_b+.yaml"
        model_cfg = "sam2.1/sam2.1_hiera_l.yaml"

        config_dir = "/home/avalocal/thesis23/KD/sam2/sam2/configs"
        with initialize_config_dir(version_base=None, config_dir=config_dir):
            cfg = compose(config_name=model_cfg)
            self.sam2_model = build_sam2(model_cfg, sam2_checkpoint, device=DEVICE)
            self.sam2_predictor = SAM2ImagePredictor(self.sam2_model)
            self.sam2_predictor.model.sam_mask_decoder.train(True)
            self.sam2_predictor.model.sam_prompt_encoder.train(True)
        
    def forward(self, x, bbox):
        '''
        input: x | should be np.array with size (h,w,3)
        bbox: array w size (n, 4) where n is the number of bounding boxes
        '''

        # print(x.shape, bbox.shape)

        # print(bbox, "this is bbox")
        self.sam2_predictor.set_image(x)
        # self.sam2_predictor.set_image_batch(x)
        

        # prompt encoding
        mask_input, unnorm_coords, labels, unnorm_box = self.sam2_predictor._prep_prompts(
            None, None, box=bbox, mask_logits=None, normalize_coords=True)

        if unnorm_coords is not None:
            concat_points = (unnorm_coords, labels)
        else:
            concat_points = None

        if unnorm_box is not None:
            box_coords = unnorm_box.reshape(-1, 2 , 2) #n,4 -> n,2,2
            box_labels = torch.tensor([[2, 3]], dtype=torch.int, device=unnorm_box.device)
            box_labels = box_labels.repeat(unnorm_box.size(0), 1)
            # we merge "boxes" and "points" into a single "concat_points" input (where
            # boxes are added at the beginning) to sam_prompt_encoder
            if concat_points is not None:
                concat_coords = torch.cat([box_coords, concat_points[0]], dim=1)
                concat_labels = torch.cat([box_labels, concat_points[1]], dim=1)
                concat_points = (concat_coords, concat_labels)
            else:
                concat_points = (box_coords, box_labels)

        
        sparse_embeddings, dense_embeddings = self.sam2_predictor.model.sam_prompt_encoder(
            points=concat_points,
            boxes = None,
            masks = mask_input,
        )



        # mask decoder

        batched_mode = (
            concat_points is not None and concat_points[0].shape[0] > 1
        )  # multi object prediction
        high_res_features = [
            feat_level[-1].unsqueeze(0)
            for feat_level in self.sam2_predictor._features["high_res_feats"]
        ]
        low_res_masks, iou_predictions, _, _ = self.sam2_predictor.model.sam_mask_decoder(
            image_embeddings=self.sam2_predictor._features["image_embed"][-1].unsqueeze(0),
            image_pe=self.sam2_predictor.model.sam_prompt_encoder.get_dense_pe(),
            sparse_prompt_embeddings=sparse_embeddings,
            dense_prompt_embeddings=dense_embeddings,
            multimask_output=False,
            repeat_image=batched_mode,
            high_res_features=high_res_features,
        )

        # Upscale the masks to the original image resolution
        masks = self.sam2_predictor._transforms.postprocess_masks(
            low_res_masks, self.sam2_predictor._orig_hw[-1]
        )
        
        return masks


# criterion = MultiStepMultiMasksAndIous(weight_dict={"loss_mask": 20.0, "loss_dice": 1.0, "loss_iou": 1.0})
# scaler = torch.cuda.amp.GradScaler()
# model


In [5]:
#train loop
from tqdm import tqdm
import torch.nn.functional as F

def train(model, train_loader, optimizer, scheduler, DEVICE, epoch):

    model.train()
    running_loss = 0.0
    for i, batch in enumerate(tqdm(train_loader)):

        torch.cuda.empty_cache()
        
        img, instance, ids, bbox, num_objects, file_name = batch
        # print(img.shape, instance.shape, ids.shape, bbox.shape, num_objects, file_name)
        #([4, 3, 1024, 2048]) ([4, 100, 1024, 2048]) ([4, 100]) ([4, 100, 4]) ([19, 50]) ('weimar_000091_000019_leftImg8bit.png',)
        # img = img.to(DEVICE)
        # instance = instance.to(DEVICE)
        # bbox = bbox.to(DEVICE)
        #resize image to 1024x1024


        img *= 255.0 
        img = F.interpolate(img, size=(1024, 1024), mode='bilinear', align_corners=False) #4, 3, 1024, 1024
        img = img[0].permute(1, 2, 0).numpy().copy().astype(np.uint8)


        # #if batch>1
        # imgList = [img[i].permute(1, 2, 0).numpy().copy().astype(np.uint8) for i in range(img.shape[0])]
        # bboxList =[bbox[i].numpy().copy().astype(np.float16) for i in range(bbox.shape[0])]
        # print(imgList[0].shape, bboxList[0].shape, "this should be input")

        # for bbox in bboxList:
        #     bbox[:, [0, 2]] = bbox[:, [0, 2]] * 1024 / 2048
        #     bbox[:, [1, 3]] = bbox[:, [1, 3]] * 1024 / 1024



        bbox = bbox[0][:num_objects[0]].numpy()
        bbox[:, [0, 2]] = bbox[:, [0, 2]] * 1024 / 2048
        bbox[:, [1, 3]] = bbox[:, [1, 3]] * 1024 / 1024
        # bbox = bbox.astype(np.float16)
        # print(bbox.shape, "this should be bbox")

        #resize instance from 1, 100, 1024, 2048 to 1, 100, 1024, 1024
        instance = F.interpolate(instance, size=(1024, 1024), mode='nearest') #4, 100, 1024, 1024
        instance = instance[0][:num_objects[0]]#.numpy().astype(np.float32)
        instance = instance.to(DEVICE)

        # print(img.shape, bbox.shape, instance.shape, "this should be input") #(1024, 1024, 3) (27, 4) torch.Size([27, 1024, 1024])

        optimizer.zero_grad()



        # print(img.shape, bbox.shape, instance.shape, "this should be input") #(1024, 1024, 3) (27, 4) torch.Size([27, 1024, 1024])
        # print(img.dtype, bbox.dtype, "this should be input dtype")           #uint8 float16 this should be input dtype

        # masks, scores, logits = model(img, bbox)
        with torch.cuda.amp.autocast():
            # print(img.shape, bbox.shape, "this should be input")

            if bbox.shape[0] == 0:
                # masks = torch.zeros((0, 1024, 1024), dtype=torch.float32, device=DEVICE)
                continue
            else:

                # masks= model(imgList, bboxList)
                masks = model(img, bbox)
            # print(masks.shape, masks.type(), masks.dtype, masks.requires_grad, "this should be output")
            #torch.Size([12, 1, 1024, 1024]) torch.cuda.FloatTensor torch.float32 True this should be output

            #masks numpy array to torch tensor
            masks = masks.squeeze(1) #24, 1024, 1024
        
        #does masks require grad?
        # print(masks.requires_grad, "this should be True")
        # print(masks.shape, instance.shape, num_objects[0], "this should be input")
        

        #loss = 20 * focal loss + 1 * dice loss 
        # print(masks.shape, instance.shape, num_objects[0], "this should be input")
        # if masks.shape != instance.shape:


        focalLoss = sigmoid_focal_loss(masks, instance, num_objects[0], loss_on_multimask=False)
        diceLoss = dice_loss(masks, instance, num_objects[0], loss_on_multimask=False)
        # print(focalLoss, diceLoss, "this should be loss")
        loss = 20 * focalLoss + 1 * diceLoss

        # print(loss, "this should be loss")
        # print(loss.requires_grad, "this should be True")
        # print("type and dtype of loss", loss.type(), loss.dtype)
        
        #backpropagation
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        running_loss += loss.item()

    return running_loss/len(train_loader)

num_epochs = 50

model = SAM2Model()
model = model.to(DEVICE)
# model = nn.DataParallel(model)
##image_encoder, memory_attention, memory_encoder, sam_prompt_encoder, sam_mask_decoder
#freeze all layers except sam_prompt_encoder and sam_mask_decoder
for name, param in model.named_parameters():
    if 'sam_prompt_encoder' in name or 'sam_mask_decoder' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False
    
optimizer = optim.AdamW(model.parameters(), lr=5.0e-6, weight_decay=1e-4, betas=(0.9, 0.999))
steps_per_epoch = len(train_loader)
scheduler = optim.lr_scheduler.PolynomialLR(optimizer, steps_per_epoch * num_epochs, power=1.0) #mask2former paper

        

#train loop
min_loss = float('inf')

for epoch in range(num_epochs):

    torch.cuda.empty_cache()

    train_loss = train(model, train_loader, optimizer, scheduler, DEVICE, epoch)
    print(f'Epoch: {epoch+1}, Train Loss: {train_loss}')

    if epoch > 40 and train_loss < min_loss:
        min_loss = train_loss
        #save model
        torch.save(model.state_dict(), '/home/avalocal/thesis23/KD/sam2/checkpoints/sam2.1_hiera_large_finetuned_instance.pth')
        print("model saved")
        

    






100%|██████████| 83/83 [01:15<00:00,  1.10it/s]


Epoch: 0, Train Loss: 18.447716997330446


100%|██████████| 83/83 [01:17<00:00,  1.07it/s]


Epoch: 1, Train Loss: 5.914421885846609


100%|██████████| 83/83 [01:17<00:00,  1.08it/s]


...

Epoch: 47, Train Loss: 2.5268445216029525
model saved


100%|██████████| 83/83 [01:12<00:00,  1.14it/s]


Epoch: 48, Train Loss: 2.525430271424443
model saved


100%|██████████| 83/83 [01:12<00:00,  1.15it/s]


Epoch: 49, Train Loss: 2.5244839076536247
model saved


In [ ]:
# #train loop with lr 1e-5
# from tqdm import tqdm
# import torch.nn.functional as F

# def train(model, train_loader, optimizer, scheduler, DEVICE, epoch):

#     model.train()
#     running_loss = 0.0
#     for i, batch in enumerate(tqdm(train_loader)):

#         torch.cuda.empty_cache()
        
#         img, instance, ids, bbox, num_objects, file_name = batch
#         # print(img.shape, instance.shape, ids.shape, bbox.shape, num_objects, file_name)
#         #([1, 3, 1024, 2048]) ([1, 100, 1024, 2048]) ([1, 100]) ([1, 100, 4]) ([19]) ('weimar_000091_000019_leftImg8bit.png',)

#         #resize image to 1024x1024
#         img *= 255.0
#         img = F.interpolate(img, size=(1024, 1024), mode='bilinear', align_corners=False) #1, 3, 1024, 2048
#         img = img[0].permute(1, 2, 0).numpy().copy().astype(np.uint8)

#         bbox = bbox[0][:num_objects[0]].numpy()
#         bbox[:, [0, 2]] = bbox[:, [0, 2]] * 1024 / 2048
#         bbox[:, [1, 3]] = bbox[:, [1, 3]] * 1024 / 1024
#         # bbox = bbox.astype(np.float16)
#         # print(bbox.shape, "this should be bbox")

#         #resize instance from 1, 100, 1024, 2048 to 1, 100, 1024, 1024
#         instance = F.interpolate(instance, size=(1024, 1024), mode='nearest')
#         instance = instance[0][:num_objects[0]]#.numpy().astype(np.float32)
#         instance = instance.to(DEVICE)

#         optimizer.zero_grad()


#         # print(img.shape, bbox.shape, instance.shape, "this should be input") #(1024, 1024, 3) (27, 4) torch.Size([27, 1024, 1024])
#         # print(img.dtype, bbox.dtype, "this should be input dtype")           #uint8 float16 this should be input dtype

#         # masks, scores, logits = model(img, bbox)
#         with torch.cuda.amp.autocast():
#             # print(img.shape, bbox.shape, "this should be input")

#             if bbox.shape[0] == 0:
#                 # masks = torch.zeros((0, 1024, 1024), dtype=torch.float32, device=DEVICE)
#                 continue
#             else:

#                 masks= model(img, bbox) #(9, 1, 1024, 1024) (9, 1) (9, 1, 256, 256) this should be output

#             # print(masks.shape, masks.type(), masks.dtype, masks.requires_grad, "this should be output")
#             #torch.Size([12, 1, 1024, 1024]) torch.cuda.FloatTensor torch.float32 True this should be output

#             #masks numpy array to torch tensor
#             masks = masks.squeeze(1) #24, 1024, 1024
        
#         #does masks require grad?
#         # print(masks.requires_grad, "this should be True")
#         # print(masks.shape, instance.shape, num_objects[0], "this should be input")
        

#         #loss = 20 * focal loss + 1 * dice loss 
#         # print(masks.shape, instance.shape, num_objects[0], "this should be input")
#         # if masks.shape != instance.shape:


#         focalLoss = sigmoid_focal_loss(masks, instance, num_objects[0], loss_on_multimask=False)
#         diceLoss = dice_loss(masks, instance, num_objects[0], loss_on_multimask=False)
#         # print(focalLoss, diceLoss, "this should be loss")
#         loss = 20 * focalLoss + 1 * diceLoss

#         # print(loss, "this should be loss")
#         # print(loss.requires_grad, "this should be True")
#         # print("type and dtype of loss", loss.type(), loss.dtype)
        
#         #backpropagation
#         loss.backward()
#         optimizer.step()
#         scheduler.step()
        
#         running_loss += loss.item()

#     return running_loss/len(train_loader)

# num_epochs = 50

# model = SAM2Model()
# model = model.to(DEVICE)
# ##image_encoder, memory_attention, memory_encoder, sam_prompt_encoder, sam_mask_decoder
# #freeze all layers except sam_prompt_encoder and sam_mask_decoder
# for name, param in model.named_parameters():
#     if 'sam_prompt_encoder' in name or 'sam_mask_decoder' in name:
#         param.requires_grad = True
#     else:
#         param.requires_grad = False
    
# optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.1, betas=(0.9, 0.999))
# steps_per_epoch = len(train_loader)
# scheduler = optim.lr_scheduler.PolynomialLR(optimizer, steps_per_epoch * num_epochs, power=1.0) #mask2former paper

        

# #train loop
# min_loss = float('inf')

# for epoch in range(num_epochs):

#     torch.cuda.empty_cache()

#     train_loss = train(model, train_loader, optimizer, scheduler, DEVICE, epoch)
#     print(f'Epoch: {epoch}, Train Loss: {train_loss}')

#     if epoch > 40 and train_loss < min_loss:
#         min_loss = train_loss
#         #save model
#         # torch.save(model.state_dict(), '/home/avalocal/thesis23/KD/sam2/checkpoints/sam2.1_hiera_large_finetuned_instance_1e-5.pth')
#         # print("model saved")
#         torch.save({"model": model.state_dict()}, '/home/avalocal/thesis23/KD/sam2/checkpoints/sam2.1_hiera_large_finetuned_instance_1e-5.pth')

#         # sd = torch.load(ckpt_path, map_location="cpu", weights_only=True)["model"]

        

    






In [ ]:
# #train loop with lr 5e-5
# from tqdm import tqdm
# import torch.nn.functional as F

# def train(model, train_loader, optimizer, scheduler, DEVICE, epoch):

#     model.train()
#     running_loss = 0.0
#     for i, batch in enumerate(tqdm(train_loader)):

#         torch.cuda.empty_cache()
        
#         img, instance, ids, bbox, num_objects, file_name = batch
#         # print(img.shape, instance.shape, ids.shape, bbox.shape, num_objects, file_name)
#         #([1, 3, 1024, 2048]) ([1, 100, 1024, 2048]) ([1, 100]) ([1, 100, 4]) ([19]) ('weimar_000091_000019_leftImg8bit.png',)

#         #resize image to 1024x1024
#         img *= 255.0
#         img = F.interpolate(img, size=(1024, 1024), mode='bilinear', align_corners=False) #1, 3, 1024, 2048
#         img = img[0].permute(1, 2, 0).numpy().copy().astype(np.uint8)

#         bbox = bbox[0][:num_objects[0]].numpy()
#         bbox[:, [0, 2]] = bbox[:, [0, 2]] * 1024 / 2048
#         bbox[:, [1, 3]] = bbox[:, [1, 3]] * 1024 / 1024
#         # bbox = bbox.astype(np.float16)
#         # print(bbox.shape, "this should be bbox")

#         #resize instance from 1, 100, 1024, 2048 to 1, 100, 1024, 1024
#         instance = F.interpolate(instance, size=(1024, 1024), mode='nearest')
#         instance = instance[0][:num_objects[0]]#.numpy().astype(np.float32)
#         instance = instance.to(DEVICE)

#         optimizer.zero_grad()


#         # print(img.shape, bbox.shape, instance.shape, "this should be input") #(1024, 1024, 3) (27, 4) torch.Size([27, 1024, 1024])
#         # print(img.dtype, bbox.dtype, "this should be input dtype")           #uint8 float16 this should be input dtype

#         # masks, scores, logits = model(img, bbox)
#         with torch.cuda.amp.autocast():
#             # print(img.shape, bbox.shape, "this should be input")

#             if bbox.shape[0] == 0:
#                 # masks = torch.zeros((0, 1024, 1024), dtype=torch.float32, device=DEVICE)
#                 continue
#             else:

#                 masks= model(img, bbox) #(9, 1, 1024, 1024) (9, 1) (9, 1, 256, 256) this should be output

#             # print(masks.shape, masks.type(), masks.dtype, masks.requires_grad, "this should be output")
#             #torch.Size([12, 1, 1024, 1024]) torch.cuda.FloatTensor torch.float32 True this should be output

#             #masks numpy array to torch tensor
#             masks = masks.squeeze(1) #24, 1024, 1024
        
#         #does masks require grad?
#         # print(masks.requires_grad, "this should be True")
#         # print(masks.shape, instance.shape, num_objects[0], "this should be input")
        

#         #loss = 20 * focal loss + 1 * dice loss 
#         # print(masks.shape, instance.shape, num_objects[0], "this should be input")
#         # if masks.shape != instance.shape:


#         focalLoss = sigmoid_focal_loss(masks, instance, num_objects[0], loss_on_multimask=False)
#         diceLoss = dice_loss(masks, instance, num_objects[0], loss_on_multimask=False)
#         # print(focalLoss, diceLoss, "this should be loss")
#         loss = 20 * focalLoss + 1 * diceLoss

#         # print(loss, "this should be loss")
#         # print(loss.requires_grad, "this should be True")
#         # print("type and dtype of loss", loss.type(), loss.dtype)
        
#         #backpropagation
#         loss.backward()
#         optimizer.step()
#         scheduler.step()
        
#         running_loss += loss.item()

#     return running_loss/len(train_loader)

# num_epochs = 50

# model = SAM2Model()
# model = model.to(DEVICE)
# ##image_encoder, memory_attention, memory_encoder, sam_prompt_encoder, sam_mask_decoder
# #freeze all layers except sam_prompt_encoder and sam_mask_decoder
# for name, param in model.named_parameters():
#     if 'sam_prompt_encoder' in name or 'sam_mask_decoder' in name:
#         param.requires_grad = True
#     else:
#         param.requires_grad = False
    
# optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.1, betas=(0.9, 0.999))
# steps_per_epoch = len(train_loader)
# scheduler = optim.lr_scheduler.PolynomialLR(optimizer, steps_per_epoch * num_epochs, power=1.0) #mask2former paper

        

# #train loop
# min_loss = float('inf')

# for epoch in range(num_epochs):

#     torch.cuda.empty_cache()

#     train_loss = train(model, train_loader, optimizer, scheduler, DEVICE, epoch)
#     print(f'Epoch: {epoch}, Train Loss: {train_loss}')

#     if epoch > 40 and train_loss < min_loss:
#         min_loss = train_loss
#         #save model
#         torch.save({"model": model.state_dict()}, '/home/avalocal/thesis23/KD/sam2/checkpoints/sam2.1_hiera_large_finetuned_instance_5e-5.pth')
#         print("model saved")
        

    






100%|██████████| 2975/2975 [12:16<00:00,  4.04it/s]


Epoch: 0, Train Loss: 4.07701341015952


100%|██████████| 2975/2975 [12:07<00:00,  4.09it/s]


Epoch: 1, Train Loss: 3.6322245692104853


100%|██████████| 2975/2975 [12:05<00:00,  4.10it/s]


...

Epoch: 47, Train Loss: 2.0926025253534317
model saved


100%|██████████| 2975/2975 [12:06<00:00,  4.09it/s]


Epoch: 48, Train Loss: 2.0834104298743883
model saved


100%|██████████| 2975/2975 [12:04<00:00,  4.11it/s]


Epoch: 49, Train Loss: 2.075776378958165
model saved
